<a href="https://colab.research.google.com/github/abhishekj98/PropertyInspectionPrediction/blob/master/GratefulDead_CheekyEdition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
pd.__version__

u'0.22.0'

In [0]:
#!pip install spacy

In [0]:
import numpy as np
import io
from nltk import ngrams


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'1CUaLwXYDAjmIZsvS27bhbp0N0kwt4_5y'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv')  


In [0]:
#3. Read file as panda dataframe
df = pd.read_csv('train.csv') 

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df2 = df.copy()


In [0]:
df2 = df2.dropna()

df2['label'] = df2['label'].apply(lambda x : int(x))
df2['text'] = df2['text'].apply(lambda x : x.lower().rstrip().replace('\n', ' ').replace('\\\'', '\''))

"""train_frac = 0.8
train = df2.sample(frac = train_frac, random_state = 4269)
test = df2.drop(train.index)

train = train.reset_index()
test = test.reset_index()"""




'train_frac = 0.8\ntrain = df2.sample(frac = train_frac, random_state = 4269)\ntest = df2.drop(train.index)\n\ntrain = train.reset_index()\ntest = test.reset_index()'

In [0]:
df2 = df2.sample(frac=1).reset_index(drop=True)
train_ind = int(1*len(df2))
train_ind


400000

In [0]:
#train['one'] =  train['text'].apply(lambda x : list(ngrams(x.split(), 1)))

In [0]:
#train['two'] =  train['text'].apply(lambda x : list(ngrams(x.split(), 2)))


In [0]:
#train['three'] =  train['text'].apply(lambda x : list(ngrams(x.split(), 3)))


In [0]:
corpus = []
#X = []
Y = []
exclaim = []
#question = []
for i in xrange(len(df2)):
  #X.insert(i, train['one'][i] + train['two'][i])
  Y.insert(i, [df2['label'][i]])
  corpus.insert(i, df2['text'][i])
  exclaim.insert(i, [1 if df2['text'][i].find('!') else 0])
  #question.insert(i, [1 if df2['text'][i].find('?') else 0])

#X = np.array(X)
#Y = np.array(Y)

In [0]:
for i in xrange(20):
  print corpus[i]
  print Y[i]

great hole in the wall. there are a lot of options on this street, most that don't look all that promising. this place was in a strip mall and hard to find.   food was awesome, we got the wings and fries. we got the pint special, so a free beer with this wasn't a bad deal at all.   the bartender was a great guy. might want to speak above a whisper in the future, but not a big deal.
[2]
i was involved in an accident where i was evidently not at fault. sent them photos to prove it as well. not only they condemned me to being partially at fault and destroyed my clean driving records, but after 10 years of being their royal costumer, they totally disregarded their "no premium change after first at fault accident" and increased my premium by 2.5 times.
[0]
from a comparison point of view, this restaurant is not in the same league as the fogo in las vegas. the quality of the food was true to fago's high standards but the service was over-the-top-annoying. there was way too much "is everythin

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
#import nltk
#nltk.download('stopwords') 
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=['the', 'a'])
#, stop_words=set(nltk.corpus.stopwords.words('english')))
vectorizer

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=['the', 'a'],
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [0]:
vect3 = vectorizer.fit_transform(corpus)
vect3

<400000x4962217 sparse matrix of type '<type 'numpy.int64'>'
	with 64538441 stored elements in Compressed Sparse Row format>

In [0]:
from scipy.sparse import hstack, csr_matrix
vect2 = hstack([vect3, exclaim])
#vect = hstack([vect2, question])
vect = csr_matrix(vect2)
vect

<400000x4962218 sparse matrix of type '<type 'numpy.int64'>'
	with 64938430 stored elements in Compressed Sparse Row format>

In [0]:
from sklearn import svm

lin_clf = svm.LinearSVC(class_weight='balanced', multi_class='ovr', max_iter=2000, )
lin_clf.fit(vect[:train_ind], np.ravel(Y[:train_ind]))

In [0]:
"""from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

y_pred = lin_clf.predict(vect[train_ind:])
y_test = Y[train_ind:]
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))  """ 

'from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score\n\ny_pred = lin_clf.predict(vect[train_ind:])\ny_test = Y[train_ind:]\nprint(f1_score(y_test, y_pred, average="macro"))\nprint(precision_score(y_test, y_pred, average="macro"))\nprint(recall_score(y_test, y_pred, average="macro"))  '

In [0]:
"""from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)"""

'from sklearn.metrics import confusion_matrix\nconfusion_matrix(y_test, y_pred)'

In [0]:
downloaded = drive.CreateFile({'id':'1-nARzPoOjxSAFcbvv_eMeWvNs86ZaeUI'}) # replace the id with id of file you want to access
downloaded.GetContentFile('test_set.csv')  
df_ftest = pd.read_csv('test_set.csv') 

NameError: ignored

In [0]:
vectorizer.vocabulary_

NameError: ignored

In [0]:
vectorizer2 = CountVectorizer(ngram_range=(1, 2), stop_words=['the', 'a'], vocabulary=vectorizer.vocabulary_)


NameError: ignored

In [0]:

df_ftest['text'] = df_ftest['text'].apply(lambda x : x.lower().rstrip().replace('\n', ' ').replace('\\\'', '\''))

In [0]:
corpus_t = []
#X = []
exclaim_t = []
#question = []
for i in xrange(len(df_ftest)):
  #X.insert(i, train['one'][i] + train['two'][i])
  corpus_t.insert(i, df_ftest['text'][i])
  exclaim_t.insert(i, [1 if df_ftest['text'][i].find('!') else 0])
  
len(df_ftest)

NameError: ignored

In [0]:
vect_t = vectorizer2.fit_transform(corpus_t)
vect_t

NameError: ignored

In [0]:
vect_t2 = hstack([vect_t, exclaim_t])
#vect = hstack([vect2, question])
vect_t2 = csr_matrix(vect_t2)
vect_t2

NameError: ignored

In [0]:
y_t_pred = lin_clf.predict(vect_t2)


NameError: ignored

In [0]:
df_ret = pd.DataFrame(y_t_pred)
df_ret.index += 1 
df_ret['ID'] = df_ret.index
df_ret.columns = ['label', 'ID']
df_ret = df_ret[['ID', 'label']]
df_ret

NameError: ignored

In [0]:
df_ret.to_csv("result_svm_ch.csv", index=False)

NameError: ignored

In [0]:
from google.colab import files
files.download('result_svm_ch.csv')

OSError: ignored